# Make-o-Matic Gesture Recognition

## Part 3: Machine Learning

2017 by Thomas Lidy, TU Wien

### Requirements

Python 2.7

pip install -r requirements.txt

Tested on OS: Ubuntu 16.04.3 LTS

In [29]:
import numpy as np
import pandas as pd
import json
import time # for time measuring
import datetime # for time printing

from scipy import stats
from scipy.signal import resample
# Power spectral density using a periodogram
from scipy.signal import periodogram

from collections import Counter # for majority vote
from collections import OrderedDict # for color palette

# Machine Learning
from sklearn import preprocessing, svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [30]:
def str_to_int(string):
    '''cut away first character and convert to int - used to convert Gesture IDs like "G01" to 1'''
    return int(string[1:])

In [31]:
def timestr(seconds):
    ''' returns HH:MM:ss formatted time string for given seconds
    (seconds can be a float with milliseconds included, but only the integer part will be used)
    :return: string
    '''
    return str(datetime.timedelta(seconds=int(seconds)))

## Read Meta-Data

In [32]:
# main data

# original input
#csv_file = 'data/EXPORT_09042017173622.csv'

# preprocessed input
csv_file = 'data/EXPORT_09042017173622_preprocessed.csv'


# json files to translate gestures, parcours into long text
#gestures_file = 'data/gestures.json' # this is the file edited manually by us to conform to json
gestures_file = 'data/gestures.json.orig' # this is the file edited manually by us to conform to json
parcours_file = 'data/parcours.json'
mutations_file = 'data/mutations.json'

files = (gestures_file, parcours_file, mutations_file)
dataframes = []

# NOTE THAT THESE JSON FILES ARE NOT JSON CONFORM
# each line is a json string on its own, so we need to process the json line by line and combine THEN into a list

In [33]:
def get_oid(oid_dict):
    # get from the original representation {u'$oid': u'589c8ed31337b5ab1e1be121'} just the oid
    return oid_dict['$oid']

In [34]:
# get meta-files with descriptions of gestures, parcours and mutations
for filename in files:
    with open(filename) as f:
        lines = [line.rstrip('\n') for line in f]   # .decode("utf-8")

    lines = [json.loads(line) for line in lines]
    
    # convert list of json lines into Dataframe
    df = pd.DataFrame.from_dict(lines)
    
    # convert long $oid to short
    df['_id'] = df['_id'].apply(get_oid)
    
    # set the real id
    df.set_index('id', inplace=True)
    
    # convert index (ID) from string like 'G01' to int
    df.index = df.index.map(str_to_int)
    
    dataframes.append(df)

In [35]:
(gestures_df, parcours_df, mutations_df) = tuple(dataframes)

In [36]:
gestures_df

,_id,isGarbage,isNesture,name,slug
1,58a23a22d826756404709446,NaN,NaN,Single Rotation klein rechtsrum,rssr
2,58a23a22d826756404709447,NaN,NaN,Single Rotation klein linksrum,rssl
3,58a23a22d826756404709448,NaN,NaN,Oszillierende Rotation klein rechtsrum,rosr
4,58a23a22d826756404709449,NaN,NaN,Oszillierende Rotation klein linksrum,rosl
5,58a23a22d82675640470944a,NaN,NaN,Single Rotation groß rechtsrum,rsbr
6,58a23a22d82675640470944b,NaN,NaN,Single Rotation groß linksrum,rsbl
7,58a23a22d82675640470944c,NaN,NaN,Oszillierende Rotation groß rechtsrum,robr
8,58a23a22d82675640470944d,NaN,NaN,Oszillierende Rotation groß linksrum,robl
9,58a23a22d82675640470944e,NaN,NaN,Kontinuierliche Rotation groß rechtsrum,rcbr
10,58a23a22d82675640470944f,NaN,NaN,Kontinuierliche Rotation groß linksrum,rcbl


In [37]:
# "positive" gestures to recognize (not nestures)
gestures_pos = gestures_df[gestures_df['isNesture'] != True].index.tolist()
gestures_pos

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [38]:
# "negative" gestures (nestures)
gestures_neg = gestures_df[gestures_df['isNesture'] == True].index.tolist()
nestures = gestures_neg # synonym
gestures_neg

[14, 15, 16, 17, 18]

#### Define handy function shortcut

In [39]:
def gesture_name(gesture_id):
    if gesture_id is None: return None
    return gestures_df.loc[gesture_id,'name']

## Read Experiment Data

In [40]:
# Experiment Data
data = pd.read_csv(csv_file)
data.head(10)

,Trainset,Experiment,Subject,TimeStamp,RFID,GRASP_A,GRASP_B,GRASP_C,AX,AY,AZ,EX,EY,EZ,Parcours,Parcours_Step,Mutation,Host,Host/Spot,Gesture
0,_TRAINSET14022017094616,1,Andreas,0,000000000000,781,8,797,0.06,-0.02,-0.10,216.8125,9.0625,-81.9375,101,1,151,8,NaN,15
1,_TRAINSET14022017094616,1,Andreas,29001,000000000000,782,0,799,0.09,-0.04,-0.11,217.0625,9.0625,-81.9375,101,1,151,8,NaN,15
2,_TRAINSET14022017094616,1,Andreas,46136,000000000000,782,6,798,0.12,-0.09,0.09,217.4375,9.1250,-81.8750,101,1,151,8,NaN,15
3,_TRAINSET14022017094616,1,Andreas,74902,000000000000,784,7,798,0.08,-0.08,0.03,217.6250,9.1250,-81.8125,101,1,151,8,NaN,15
4,_TRAINSET14022017094616,1,Andreas,97663,000000000000,781,0,798,0.07,-0.09,0.04,217.9375,9.1875,-81.7500,101,1,151,8,NaN,15
5,_TRAINSET14022017094616,1,Andreas,116448,000000000000,784,4,800,0.12,-0.06,-0.03,218.3125,9.2500,-81.7500,101,1,151,8,NaN,15
6,_TRAINSET14022017094616,1,Andreas,148753,000000000000,783,0,798,0.21,-0.04,0.03,218.5000,9.3125,-81.7500,101,1,151,8,NaN,15
7,_TRAINSET14022017094616,1,Andreas,167422,000000000000,784,2,798,0.18,-0.10,-0.08,218.6875,9.3750,-81.7500,101,1,151,8,NaN,15
8,_TRAINSET14022017094616,1,Andreas,187481,000000000000,782,4,799,0.15,-0.18,-0.03,219.0000,9.4375,-81.7500,101,1,151,8,NaN,15
9,_TRAINSET14022017094616,1,Andreas,213733,000000000000,784,13,799,0.15,-0.18,-0.17,219.1250,9.4375,-81.7500,101,1,151,8,NaN,15


In [41]:
# get the TimeStamp Diffs 
# (TimeStamps are reset after each Parcours, so we have to do it groupwise by Parcours)
group_by = ('Subject','Experiment','Trainset','Parcours')
timestamp_deltas = data.groupby(group_by)['TimeStamp'].diff()
#timestamp_deltas

In [42]:
timestamp_deltas.describe()

count    7.108210e+05
mean     2.586989e+04
std      3.509427e+04
min      1.930000e+02
25%      1.757400e+04
50%      2.400500e+04
75%      2.863500e+04
max      5.743871e+06
dtype: float64

In [43]:
# Note: Not sure why there is a value of 5,743,871 as the max timestamp delta

### Time Stamp delta and Sample Rate of Signal

In [44]:
time_delta = timestamp_deltas.median()
print "Time stamp deltas are on median", time_delta, "micro-seconds"

Time stamp deltas are on median 24005.0 micro-seconds


In [45]:
time_delta_sec = time_delta / 1000000  # microsec to sec
time_delta_sec

0.024005

In [46]:
sampling_rate = 1 /  time_delta_sec   
print "Signal sampling rate is on avg.", sampling_rate, "Hz"

Signal sampling rate is on avg. 41.6579879192 Hz


### Iterate through the data

In [47]:
# Note: Pandas group_by can be used instead of a for loop, to loop over groups of objects in the data, 
# e.g. all data in a parcours

In [48]:
# see gestures per Parcours
group_by = ('Subject','Experiment','Trainset','Parcours')
data.groupby(group_by)['Gesture'].unique()

Subject  Experiment  Trainset                 Parcours
Alfred   2           _TRAINSET14022017144824  101              [15, 1, 17]
                     _TRAINSET14022017144923  102              [15, 2, 17]
                     _TRAINSET14022017145122  103               [15, 1, 2]
                     _TRAINSET14022017145237  104               [15, 2, 1]
                     _TRAINSET14022017145434  107              [15, 1, 17]
                     _TRAINSET14022017145514  108              [15, 2, 17]
                     _TRAINSET14022017145629  109               [15, 1, 2]
                     _TRAINSET14022017145751  110               [15, 2, 1]
                     _TRAINSET14022017145913  113              [15, 1, 17]
                     _TRAINSET14022017145944  114              [15, 2, 17]
                     _TRAINSET14022017150026  115               [15, 1, 2]
                     _TRAINSET14022017150110  116               [15, 2, 1]
                     _TRAINSET14022017150614 

In [49]:
# see gestures per Parcours Step
group_by = ('Subject','Experiment','Trainset','Parcours','Parcours_Step')
data.groupby(group_by)['Gesture'].unique()

Subject  Experiment  Trainset                 Parcours  Parcours_Step
Alfred   2           _TRAINSET14022017144824  101       1                [15]
                                                        2                 [1]
                                                        3                [17]
                                                        4                 [1]
                                                        5                [17]
                                                        6                 [1]
                                                        7                [17]
                                                        8                 [1]
                                                        9                [17]
                                                        10                [1]
                     _TRAINSET14022017144923  102       1                [15]
                                                        2               

## Replace Nestures

choose whether to replace, i.e. merge nestures by their closest (following) gestures

In [50]:
replace_nestures = False

In [ ]:
group_by = ('Subject','Experiment','Trainset','Parcours','Parcours_Step','Mutation','Gesture')
group_df = data.groupby(group_by)
print "Originally", len(group_df), "individual gesture blocks"

Originally 5169 individual gesture blocks


In [ ]:
data.groupby(group_by).count().head(20)

In [ ]:
# Therefore Group by PARCOURS
# group data nicely, subdivided by Subject, Experiment, Trainset, Parcours
group_by = ('Subject','Experiment','Trainset','Parcours')

In [ ]:
# Step 1: replace ALL Nestures by NaN
if replace_nestures:
    # make a copy of the complete data before altering anything
    data_nonest = data.copy()
    idx_nestures = data_nonest['Gesture'].isin(nestures)
    # replace nestures by NaN
    data_nonest.loc[idx_nestures,'Gesture'] = np.nan
    print data_nonest.head()

In [ ]:
# now we can use the Forward FILL and Backward FILL methods of Pandas
# to replace the NaNs by the values that come before or after

# BUT: we shall not do that across Parcours/Experiments!
# GROUPBY helps us here to apply the fill methods only within a PARCOURS

if replace_nestures:
    # BACKWARD FILL first by later values to NaNs before
    data_nonest = data_nonest.groupby(group_by).bfill()

    # in case there would be NaNs left, do also a FORWARD FILL
    #data = data.groupby(group_by).ffill()
    
    print "Replaced Nestures by filling with neighboured Gestures!"
    print np.isnan(data_nonest['Gesture']).sum(), "NaN values remaining. Should be 0."
    # NOTE: bfill applies to ALL COLUMNS! so there might be other columns affected by this!
    # TODO double-check any side effects!
    
    # adding NaNs cause the Gesture column to be converted from int to float
    # we convert back to int
    data_nonest['Gesture'] = data_nonest['Gesture'].astype(int)
    
    # check/verify via groupby:
    group_by = ('Subject','Experiment','Trainset','Parcours','Gesture')
    group_df = data_nonest.groupby(group_by)
    print "After nesture replacement", len(group_df), "individual gesture blocks"

In [ ]:
#group_df.size()

In [ ]:
# from here on we use data again for data_nonest

if replace_nestures:
    # keep original data in a variable
    data_orig = data
    data = data_nonest

## Data Pre-Procssing Part I

### Which Sensor Parameters to use?

In [ ]:
include_GRASP = True

if include_GRASP:
    params = ['AX', 'AY', 'AZ', 'EX', 'EY', 'EZ', 'GRASP_A', 'GRASP_B', 'GRASP_C']
else:
    params = ['AX', 'AY', 'AZ', 'EX', 'EY', 'EZ']

# TODO add RFID?

### Global Normalize?

#### Normalize Parameter columns to -1, 1

here it's done globally. if set to False, there is an option to do it locally later

In [ ]:
normalize_global = True
# normalize_global means we normalize all parameter columns at once, globally => NO LATER TREATMENT

In [ ]:
data[params].head()

In [ ]:
if normalize_global:
    # normalize to -1, 1
    data[params] = preprocessing.minmax_scale(data[params], feature_range=(-1, 1), axis=0, copy=False)

In [ ]:
data[params].head()

## Get Isolated Gestures

### Grouping for each Gesture:

### Select correct level of detail:
* a) Parcours: all same gestures of a Parcours will be concatenated together
* b) Parcours-Step: gestures inside one Parcours will be keep individual

In [ ]:
# select: 'Parcours' or 'Parcours_Step':
#level_of_detail = 'Parcours'
level_of_detail = 'Parcours-Step' 

In [ ]:
# GET INDIVIDUAL GESTURES 
# group data nicely, subdivided by Subject, Experiment, Trainset, Parcours or Parcours-Step, Gesture

if level_of_detail == 'Parcours':
    group_by = ('Subject','Experiment','Trainset','Parcours','Gesture')
elif level_of_detail == 'Parcours-Step':
    group_by = ('Subject','Experiment','Trainset','Parcours','Parcours_Step','Gesture')
else:
    raise ValueError("invalid level_of_detail")
    
group_df = data.groupby(group_by)
group_df.mean().head(100)  # mean is not meaningful here as aggregation - just to print the structure of the data

In [ ]:
print len(group_df), "individual gesture blocks"

## Get Gesture Data: 1 Block per each individual Gesture

we put each time series that belong to 1 particular gesture in a particular parcours into a dictionary,
which contains a list of such time series blocks per gesture entry in the dict

### Create Gesture Dictionary + Reduce Data to desired parameter columns

In [ ]:
# now we ITERATE nicely through group_df and get each Gesture block individually
# -> group_data will be a dataframe just for a single gesture

i=0
# dictionary containing a list of sub-datasets for each gesture, to train ML
gesture_exp_dict = {}

for name_tuple, datablock in group_df:
    i += 1
    #print str(name_tuple)
    gesture = name_tuple[-1]  # gesture is last element of tuple, as defined in group_by above
    
    # initalize empty list for the gesture if not existing in dict yet
    if gesture not in gesture_exp_dict.keys():
        gesture_exp_dict[gesture] = [] 
        
    # reduce dataframe to params columns
    datablock = datablock[params] 
    
    # add data to gesture dict
    gesture_exp_dict[gesture].append(datablock)
    
    # NOTE that group_data here still contains ALL data columns. we will redue to params later

print "DONE:", i, "gesture blocks"

#### Some Statistics about the Gesture Block data

In [ ]:
# How many data blocks = training examples do we have for each gesture
for gest in sorted(gesture_exp_dict.keys()):
    print "G", gest, '\t', len(gesture_exp_dict[gest]), "training data blocks", '\t', gesture_name(gest) 

In [ ]:
# how many data points (= samples or timesteps) does each data block have?

data_sizes = {} # collect per gesture in dict
data_sizes_total = [] # collect all in list

print "Average length per gesture:"

for gest in sorted(gesture_exp_dict.keys()):
    print "G", gest, ':\t', 
    data_sizes[gest] = []
    for datablock in gesture_exp_dict[gest]:
        size = datablock.shape[0]
        data_sizes[gest].append(size)
        data_sizes_total.append(size)
    avg_gesture_len = int(np.mean(data_sizes[gest]))
    print "% d samples \t%0.2f sec" % (avg_gesture_len, avg_gesture_len * time_delta_sec)

In [ ]:
# minimum, average and maximum sample size of gestures
gest_len_min = min(data_sizes_total)
gest_len_avg = np.mean(data_sizes_total)
gest_len_max = max(data_sizes_total)

In [ ]:
gest_len_min, gest_len_avg, gest_len_max

In [ ]:
# average data length (number of samples)
print "Average length all gestures"
print "% d samples \t%0.2f sec" % (gest_len_avg, gest_len_avg * time_delta_sec)    

In [ ]:
pd.Series(data_sizes_total).describe()

## Pre-Processing

### Low-Pass Filter

removing high frequencies (little fluctuations which are probably not relevant)

In [ ]:
# source code from https://stackoverflow.com/questions/25191620/creating-lowpass-filter-in-scipy-understanding-methods-and-units

from scipy.signal import butter, lfilter, freqz

def butter_lowpass(cutoff, fs, order=5):
    '''cutoff: cutoff frequency in Hz
    fs: sampling rate in Hz'''
    nyq = 0.5 * fs # Nyquist frequency is half the sampling rate (fs)
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

### Set Filter Settings here

In [ ]:
# Filter settings (global vars)

# filter order (see scipy: butter)
order = 1 #3 #5 #6

# sampling rate in Hz
#fs = 30.0       # fixed assumed sample rate
fs = sampling_rate   # determined before by average time delta 

# cutoff frequency of the filter in Hz
cutoff = 4 #Hz
#cutoff = 3.667 
#cutoff = 1.3
#cutoff = 0.667 
#cutoff = 0.5
#cutoff = 0.33

### Preprocessing Function

In [ ]:
def preprocess_signal(testdata, 
                      normalize=False, 
                      resampling=False, n_samples=None, timestamps=None, window='hann', 
                      filtering=False):
    
    # Min/max normalization
    # Note: to do it the fully right way, the minmax scaling should be done on all training data coherently
    # (currently its done per training block) and the same scaling values (min and max) should be reused here
    # see http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
    if normalize:
        testdata = preprocessing.minmax_scale(testdata, feature_range=(-1, 1), axis=0)
        
    # Time Resampling
    if resampling:
        
        if n_samples is None:
            # if not a FIXED number of samples is provided, the number of samples stays the same as in the input signal
            n_samples = testdata.shape[0] 
        
        if timestamps is None:
            # NO timestamp alignment, just resampling to a given number of target samples
            testdata = resample(testdata, num=n_samples, window=window)
        else:
            # if provided, we use the original timestamps to re-align the signal
            # TODO check: length of signals in testdata must match len(timestamps)
            testdata, timestamps2 = resample(testdata, num=n_samples, t=timestamps, window='hann')
        

    if filtering:
        # filter the signal block with low-pass filter
        testdata = butter_lowpass_filter(testdata, cutoff, fs, order)
        
    return testdata

## Feature Calculation

### Zero Crossing Rate

In [ ]:
def calc_zero_crossings(datablock, normalized=False):
    '''computes row-wise zerocrossings'''
    # datablock is assumed to be pandas Dataframe and to have multiple signals in the rows
    # example for 1 signal row:
    #zcr = np.signbit(signal).diff().abs().mean()
    # for multiple signal rows:
    zcr = np.signbit(datablock).astype(int).diff(axis=0).abs().mean(axis=0)
    
    if normalized:
        # divide by length of signal, otherwise it will be directly related to the size of the chosen window
        zcr = zcr / datablock.shape[0]
    return zcr

### Statistical Features

In [ ]:
# Calc statistical features

def calc_statistical_features(matrix, axis=0):

    # to define the proper output shape, we need the "other axis" of the input shape (not the one where we compute along)
    other_axis = int(not axis) 
    n_rows = matrix.shape[other_axis]
    
    result = np.zeros((n_rows,7))
    
    result[:,0] = np.mean(matrix, axis=axis)
    result[:,1] = np.var(matrix, axis=axis, dtype=np.float64) 
    result[:,2] = stats.skew(matrix, axis=axis)
    result[:,3] = np.median(matrix, axis=axis)
    result[:,4] = np.min(matrix, axis=axis)
    result[:,5] = np.max(matrix, axis=axis)
    result[:,6] = stats.kurtosis(matrix, axis=axis, fisher=False) # Matlab calculates Pearson's Kurtosis

    result[np.where(np.isnan(result))] = 0
    return result

### Power Spectrum (~ FFT)

In [ ]:
# TODO: set window size to something like 1/2 or 1/3 of the smallest gesture

def compute_power_spectrum_avg(signal_data, window_size=100):   # window_size=360
    '''signal_data: numpy array, rows: time, columns: different signals
    computes power spectrum of each signal, in a windowed manner
    aftewards the power spectrums of each window per signal are averaged'''
    
    if window_size > len(signal_data):
        raise ValueError("Window is bigger than input signal: " + str(len(signal_data)))
    
    hop_size = int(window_size / 2)

    power_spec_list = []

    pos = 0
    while pos + window_size <= len(signal_data):
        #print pos, pos+window_size
        sig_window = signal_data[pos:pos+window_size]
        # for periodogram we need to transpose to compute the signal along the right axis
        freq_axis, power_spec = periodogram(sig_window.T, fs=sampling_rate, window='hann') 
        power_spec_list.append(power_spec)
        pos += hop_size

    power_spec_array = np.array(power_spec_list)  
    power_spec_avg = np.mean(power_spec_array, axis=0)
    return power_spec_avg, freq_axis

### Function to compute All features

In [ ]:
def calc_all_features(in_data, calc_derivative=False, calc_zerocrossings=False, calc_power_spectrum=False):

    # calc statistical features
    features = calc_statistical_features(in_data, axis=0)

    # vectorize
    features = features.flatten()

    if calc_derivative:
        # calc derivative of all signals
        in_data_deriv = np.gradient(in_data, axis=0)
        # calc statistics of derivatives
        features_deriv = calc_statistical_features(in_data_deriv, axis=0)
        # vectorize
        features_deriv = features_deriv.flatten()
        # concatenate to other features
        features = np.concatenate((features,features_deriv))

    if calc_zerocrossings:
        features_zcr = calc_zero_crossings(in_data)
        features = np.concatenate((features,features_zcr))
        
    if calc_power_spectrum:
        power_spec_avg, freq_axis = compute_power_spectrum_avg(in_data)
        # combine spectrum of all signals into 1 long vector
        power_spec_avg = power_spec_avg.flatten()
        features = np.concatenate((features,power_spec_avg))

    return features

## Start Feature Calculation

### Set Options here:

In [ ]:
# OPTIONS:

# want non_gestures in result? can use option depending on replace_nestures True or False, or exclude always
exclude_non_gestures = replace_nestures  # True

# preprocessing options
use_lowpassfilter = False     # apply low-pass filter (filtering out high frequencies) - settings see above
use_normalized = True         # local normalization, per each window!
use_resampled = True          # resample time signal to common window length (see samples parameter)
# 2 choices for resampling:
# a) just re-align by given timestamps to equi-distant timestamps, but same number of samples are kept:
n_samples = None
# b) resample ALL gestures to a FIXED common number of samples given here:
#n_samples = int(gest_len_avg)   # we choose the average gesture length for the common resampled gesture length

# override use_normalized: if we already normalized globally, we should not do it again locally
if normalize_global: use_normalized = False

# feature options: # True is better for all, except Power spectrum
calc_derivative = True
calc_zerocrossings = True
calc_powerspectrum = False

In [ ]:
# we added preprocess_signal() function below, thats why we need to use the original gesture_dict as input
input_dict = gesture_exp_dict 

In [ ]:
if exclude_non_gestures:
    gestures_to_process = gestures_pos
else:
    gestures_to_process = input_dict.keys()

In [ ]:
gestures_pos

In [ ]:
# COMPUTE FEATURES
# LOOP over all gesture data to create features

# initialize feature output for training data as a list
train_list = []
train_classes_num = []

# iterate over gesture number
for gest in sorted(gestures_to_process):
    print "G", gest, ':\t', len(input_dict[gest]), "examples"
    
    # iterate over all gesture blocks for that gesture number
    for in_data in input_dict[gest]:
        #print datablock.shape, 
        
        #if use_resampled:
        #    # resampled data has already extracted the param columns
        #    in_data = datablock
        #else:
        #    # for non-resampled we have to get the relevant data columns and transpose
        #    in_data = datablock[params].T
        
        # preprocessing
        in_data = preprocess_signal(in_data, use_normalized, 
                                    use_resampled, n_samples, timestamps=None, window=None, # 'hann'
                                    filtering=use_lowpassfilter)
                
        # convert to dataframe cause we use pandas .diff() in ZCR computation
        in_data = pd.DataFrame(in_data, columns=params)

        # calculate features
        features = calc_all_features(in_data, calc_derivative, calc_zerocrossings, calc_powerspectrum)

        # append to output list
        train_list.append(features)
        
        # store class (gesture number) for these features
        train_classes_num.append(gest)

In [ ]:
print "Feature vector length:", len(features)

# Machine Learning

### Prepare Training Data

In [ ]:
print "Training data:", len(train_list), "examples"

In [ ]:
# make feature array from feature list (ALL training data)
train_data = np.array(train_list)
del train_list # not needed any longer
train_data.shape

In [ ]:
# verify if the training categories (gesture numbers) have the same length
len(train_classes_num)

### Standardize

Zero-mean unit-variance Standardization

In [ ]:
# ad-hoc scaling
# train_data = preprocessing.scale(train_data,axis=0)
# axis=0 means independently standardize each feature, otherwise (if 1) standardize each sample

In [ ]:
# we now user StandardScaler class to keep the mean and variance for later
standardizer = preprocessing.StandardScaler()
train_data = standardizer.fit_transform(train_data)

### Train/Test Set Split

In [ ]:
# split the data into train/test set

testset_size = 0.25

# sklearn >= 0.18
# use random_state to avoid that the results fluctuate randomly
splitter = StratifiedShuffleSplit(n_splits=1, test_size=testset_size, random_state=0) 
splits = splitter.split(train_data, train_classes_num)

# Note: this for loop is only executed once, if n_splits==1
for train_index, test_index in splits:
    #print "TRAIN INDEX:", train_index
    #print "TEST INDEX:", test_index
    
    # split the data
    train_set = train_data[train_index]
    test_set = train_data[test_index]
    
    # and the numeric classes (groundtruth)
    train_classes = np.array(train_classes_num)[train_index]
    test_classes = np.array(train_classes_num)[test_index]
    
    print "TRAIN SIZE:", train_set.shape
    print "TEST SIZE:", test_set.shape
    

## 1) Gesture Regonition - isolated

### ML Algorithm: SVM

Support Vector Machines

In [ ]:
# try 3 different SVM kernels
kernels = ['linear','poly','rbf']

In [ ]:
models = {}

for kernel in kernels:
    print "SVM", kernel,
    
    # TRAIN 
    start_time = time.time() # measure time

    model = OneVsRestClassifier(SVC(kernel=kernel)) #, degree=degree)) #, n_jobs=-1)  # n_jobs = n cpus, -1 = all
    # full set
    #model.fit(train_data, train_classes_num)
    # train set
    model.fit(train_set, train_classes)
    
    # store in dict
    models[kernel] = model

    end_time = time.time()
    print "Training time:", timestr(end_time - start_time)

#### Verification on Train Set (just for plausibility)

In [ ]:
# predict on train set (will reuse last model == rbf only)
pred_train = model.predict(train_set)
# print pred_train

In [ ]:
#print train_classes

In [ ]:
# Accuracy on train set (manual computation)
np.sum(pred_train == train_classes) * 1.0 / len(train_classes)

In [ ]:
# Accuracy on train set (using scikit-learn)
accuracy_score(train_classes, pred_train)

## Evaluation

### Evaluation - Overall

In [ ]:
result_ov = pd.DataFrame(index=kernels, columns=['Accuracy','Precision','Recall','F-Measure'])

In [ ]:
for k in kernels:
    # predict on TEST set
    pred_test = models[k].predict(test_set) 
    
    # Accuracy, Precision, Reacall on TEST set
    result_ov.loc[k,'Accuracy'] = accuracy_score(test_classes, pred_test)
    result_ov.loc[k,'Precision'] = precision_score(test_classes, pred_test, average='macro')
    result_ov.loc[k,'Recall'] = recall_score(test_classes, pred_test, average='macro')
    result_ov.loc[k,'F-Measure'] = f1_score(test_classes, pred_test, average='macro')

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
result_ov*100

### Evaluation - Per Gesture

In [ ]:
# manual selection which one was the best one
best_model = models['poly']
pred_test = best_model.predict(test_set) 

In [ ]:
# TODO check if the sorting of precision_score etc. is really in this order!!
labels = sorted(np.unique(test_classes))
gesture_names = [gesture_name(l) for l in labels]

In [ ]:
# nice result dataframe
columns = ['Gesture','N_train','N_test','Precision','Recall','F1']
result_df = pd.DataFrame(index=labels,columns=columns)
result_df['Gesture'] = gesture_names

In [ ]:
# number of train / test instances
values, counts = np.unique(train_classes, return_counts=True)
result_df['N_train'] = pd.Series(counts, index=values)
values, counts = np.unique(test_classes, return_counts=True)
result_df['N_test'] = pd.Series(counts, index=values)

In [ ]:
# per class evaluation
result_df['Precision'] = precision_score(test_classes, pred_test, average=None) * 100
result_df['Recall'] = recall_score(test_classes, pred_test, average=None) * 100
result_df['F1'] = f1_score(test_classes, pred_test, average=None) * 100

In [ ]:
result_df

In [ ]:
# compare average P, R and F to overall P, R and F above (same)
result_df.mean(axis=0)

In [ ]:
# Confusion Matrix
conf = confusion_matrix(test_classes, pred_test, labels=labels) # labels defines the order
labels_long = gestures_df.loc[labels,'name']
conf_df = pd.DataFrame(conf, index=labels_long, columns=labels)
conf_df

## 2) Continuous Time Series Prediction

What is our input stream?

The data of 1 trainset, because after each trainset, the TimeStamp is reset.

In [ ]:
# a) loop over each Trainset
#group_by = ('Subject','Experiment','Trainset')

# b) use Experiment as the block where we do predictions (means it includes timestamp resets!!)
group_by = ('Subject','Experiment')

group_df = data.groupby(group_by)
group_df.max().head(50) 

In [ ]:
print len(group_df), "Experiments / Trainsets"

In [ ]:
# iterate over each Trainset
i =0
for name_tuple, group_data in group_df:
    i += 1
    #print str(name_tuple)
    
    if len(name_tuple) == 3:
        subject, exp, trainset = name_tuple
    elif len(name_tuple) == 2:
        subject, exp = name_tuple
        trainset = None
    
    break # for testing we just do 1 loop
    

In [ ]:
name_tuple

In [ ]:
group_data['TimeStamp'].min()

In [ ]:
group_data['TimeStamp'].max()

In [ ]:
if len(name_tuple) == 3:
    # check if TimeStamps are monotonously increasing
    if not np.all(group_data['TimeStamp'].diff()[1:] > 0):
        raise ValueError("Time Stamps are not monotonously increasing!")

In [ ]:
# set these to None so that plot title is not shown wrongly
parcours = None
mutation = None
gesture = None

In [ ]:
# which gestures appear in this Experiment or Trainset
group_data['Gesture'].unique()

### Pre-Process the Data - Testing

the same way as it was done for training set

In [ ]:
pd.options.display.float_format = '{:,.5f}'.format

In [ ]:
# get the relevant columns out of group_data

In [ ]:
timestamps = group_data['TimeStamp'].tolist()

In [ ]:
test_gestures = group_data['Gesture'].tolist()

In [ ]:
# 9 parameters columns
testdata = group_data[params]
testdata.shape

In [ ]:
# Global Min/max normalization
# Note: to do it the fully right way, the minmax scaling should be done on all training data coherently
# (currently its done per training block) and the same scaling values (min and max) should be reused here
# see http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
# TODO store minmax_scale from training data and reapply same scaling here

if normalize_global:
    testdata = preprocessing.minmax_scale(testdata, feature_range=(-1, 1), axis=0, copy=False)

In [ ]:
# convert to dataframe cause plot needs column names
testdata = pd.DataFrame(testdata, columns=params)
testdata.head(15)

In [ ]:
# time resample

n_samples = len(timestamps)  

if use_resampled:
    # the number of samples stays the same
    # but we use the original timestamps to re-align the signal
    testdata_res, timestamps2 = resample(testdata, num=n_samples, t=timestamps)
    
    # convert to dataframe cause plot needs column names
    testdata_res = pd.DataFrame(testdata_res, columns=params)

In [ ]:
timestamps[:15]

In [ ]:
timestamps2[:15]

In [ ]:
# timestamps are now equidistant
timestamps2[1:15] - timestamps2[:14]

In [ ]:
testdata_res.head(15)

In [ ]:
# debug check whether the values have been altered -> OK
#testdata == testdata_res

In [ ]:
# overwrite testdata with testdata_res for subsequent coherent usage
#testdata = testdata_res

### Continuous Prediction

In [ ]:
# for our window_size (= signal length of input to Machine Learning)
# we take the average signal length of the trained gestures
window_size = int(gest_len_avg) # gest_len_min 
window_size

In [ ]:
# PREDICTION RESOLUTION
# how quickly do we step forward

# for now we choose half the window_size
#step_size = window_size / 2

# for speedup we choose same as window_size
step_size = window_size 

# can be set smaller for higher resolution

# TODO: set in milliseconds - convert back to sample length

step_size

In [ ]:
print "Testing in continous signal with %.3f sec window and %.3f sec step size" % (window_size * time_delta_sec, 
                                                                                  step_size * time_delta_sec)

In [ ]:
# TODO: align with preprocess_signal function used in training data above

def preprocess_signal_continuous(testdata, normalize=False, resampling=False, timestamps=None, filtering=False):
    
    # Min/max normalization
    # Note: to do it the fully right way, the minmax scaling should be done on all training data coherently
    # (currently its done per training block) and the same scaling values (min and max) should be reused here
    # see http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

    if normalize:
        testdata = preprocessing.minmax_scale(testdata, feature_range=(-1, 1), axis=0)
        
    # Time Resampling
    if resampling:
        # the number of samples stays the same
        # if provided, we use the original timestamps to re-align the signal
        n_samples = testdata.shape[0] # must match len(timestamps)
        testdata, timestamps2 = resample(testdata, num=n_samples, t=timestamps) #, window='hann')

    if filtering:
        # filter the signal block with low-pass filter
        testdata = butter_lowpass_filter(testdata, cutoff, fs, order)
        
    return testdata

In [ ]:
# PREDICTION LOOP OVER 1 TRAINING INPUT BLOCK

def continuous_prediction(testdata, test_gestures, window_size, step_size):
    pos = 0
    n_samples = testdata.shape[0]
    
    # output
    test_groundtruth = [] # we create the groundtruth to compare with here
    predictions = []  # predictions are collected here

    while pos < (n_samples - window_size):
        # 1) DETERMINE GROUNDTRUTH (for evaluation only, not used in predictions)
        
        # to get the "correct" gesture for that window, we cut the same part of the gesture information
        test_window_groundtruth = test_gestures[pos:pos+window_size]

        # HOW to determine groundtruth?
        # a) determine gesture from the window center
        gt_gesture = test_gestures[pos+(window_size/2)]
        
        # b) Majority vote: which is the predominant label in the whole window
        #gt_gesture = Counter(test_window_groundtruth).most_common()[0][0]

        # 2) DO PREDICTIONS
        
        # cut a window out of the incoming signal
        signal = testdata[pos:pos+window_size]

        # calc features
        features = calc_all_features(signal, calc_derivative, calc_zerocrossings)

        # reshape to row vector for standardize and predict below (= single input sample)
        features = features.reshape(1, -1)  
        
        # STANDARDIZE features, the same way as done in training (reusing those mean and var)
        features = standardizer.transform(features)

        # ML prediction of gesture
        pred_gesture = best_model.predict(features)[0]

        # add to groundtruth and prediction list
        test_groundtruth.append(gt_gesture)
        predictions.append(pred_gesture)

        # step forward
        pos += step_size
    
    return test_groundtruth, predictions

In [ ]:
# LOOP over ALL Experiments or Trainsets

i = 0
n_groups = len(group_df)

test_groundtruth_all = [] # we create the groundtruth to compare with here
predictions_all = []  # predictions are collected here

for name_tuple, group_data in group_df:
    
    i += 1
    print "Experiment", i, "/", n_groups, ":", str(name_tuple), group_data.shape,
    
    # just metadata
    if len(name_tuple) == 3:
        subject, exp, trainset = name_tuple
    elif len(name_tuple) == 2:
        subject, exp = name_tuple
        trainset = None
    
    # get signals, timestamps and gesture groundtruth
    timestamps = group_data['TimeStamp'].tolist()   # timestamps
    testdata = group_data[params]                   # signals
    test_gestures = group_data['Gesture'].tolist()  # groundtruth
    
    # preprocess testdata
    print "Preprocessing ...",
    testdata = preprocess_signal_continuous(testdata, use_normalized, use_resampled, timestamps, use_lowpassfilter)
    #print testdata.shape
    
    # convert to dataframe cause we use pandas .diff() in ZCR computation
    testdata = pd.DataFrame(testdata, columns=params)
    
    print "Prediction:", 
    test_groundtruth, predictions = continuous_prediction(testdata, test_gestures, window_size, step_size)
    print len(predictions), "predictions"
    
    test_groundtruth_all.extend(test_groundtruth)
    predictions_all.extend(predictions)
    

In [ ]:
print len(predictions_all), "predictions"

In [ ]:
print "collected true gestures include:"
np.unique(test_groundtruth_all).tolist()

In [ ]:
print "predicted gestures include:"
np.unique(predictions_all).tolist()

In [ ]:
pd.DataFrame({'groundt':test_groundtruth_all, 'pred':predictions_all})

In [ ]:
result_ov = pd.DataFrame(columns=['result']) #columns=['Accuracy','Precision','Recall','F-Measure'])

# Accuracy, Precision, Reacall on TEST set
result_ov.loc['Accuracy'] = accuracy_score(test_groundtruth_all, predictions_all)
result_ov.loc['Precision'] = precision_score(test_groundtruth_all, predictions_all, average='macro')
result_ov.loc['Recall'] = recall_score(test_groundtruth_all, predictions_all, average='macro')
result_ov.loc['F-Measure'] = f1_score(test_groundtruth_all, predictions_all, average='macro')
result_ov = result_ov * 100
result_ov

#### Confusion Matrix

In [ ]:
labels_long = gestures_df.loc[labels,'name']
conf = confusion_matrix(test_groundtruth_all, predictions_all, labels=labels) # labels defines the order
conf_df = pd.DataFrame(conf, index=labels_long, columns=labels)
conf_df